In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfTransformer,TfidfVectorizer
import nltk

In [2]:
test = pd.read_csv('test.csv',encoding='utf-8')
train= pd.read_csv('train.csv', encoding='utf-8')

In [3]:
print(train.shape)
print(test.shape)

(420464, 2)
(32875, 2)


In [4]:
train.head()

,url,label
0,diaryofagameaddict.com,bad
1,espdesign.com.au,bad
2,iamagameaddict.com,bad
3,kalantzis.net,bad
4,slightlyoffcenter.net,bad


In [5]:
train['label'].value_counts()

good    344821
bad      75643
Name: label, dtype: int64

In [6]:
train_x = train.iloc[:,0]
train_y = train.iloc[:,1]

In [7]:
train_x[:6]

0    diaryofagameaddict.com
1          espdesign.com.au
2        iamagameaddict.com
3             kalantzis.net
4     slightlyoffcenter.net
5          toddscarwash.com
Name: url, dtype: object

In [8]:
train_y[: 6]

0    bad
1    bad
2    bad
3    bad
4    bad
5    bad
Name: label, dtype: object

In [9]:
from sklearn.preprocessing import LabelEncoder
lbl = LabelEncoder()
train_y = lbl.fit_transform(train_y)

In [10]:
train_y[train_y==1].shape # good

(344821,)

In [11]:
train_y[train_y==0].shape # bad

(75643,)

### 分词---法①

In [149]:
# test
sent = "lazycat.I'm super lying man,This is a coooool #dummysmiley: :-) :-P <3 and some arrows < > -><--#@@qq.com"
print(nltk.word_tokenize(sent))#该方法要求被处理的字符串本身各个词语之间有空格，能处理如don't, they'll等缩写词的情况。
print(len(nltk.word_tokenize(sent)))
print(nltk.tokenize.word_tokenize(sent))#该方法要求被处理的字符串本身各个词语之间有空格，能处理如don't, they'll等缩写词的情况。
print(len(nltk.tokenize.word_tokenize(sent)))
print(nltk.tokenize.TweetTokenizer().tokenize(sent))#能够拆分无效用的标点符号
print(len(nltk.tokenize.TweetTokenizer().tokenize(sent)))
print(nltk.tokenize.TweetTokenizer(strip_handles=True, reduce_len = True).tokenize(sent))#当一个词中相同字符连续出现3次以上，就只保留3个。设置strip_handles = True会删去@xxx。
print(len(nltk.tokenize.TweetTokenizer().tokenize(sent)))

['lazycat.I', "'m", 'super', 'lying', 'man', ',', 'This', 'is', 'a', 'coooool', '#', 'dummysmiley', ':', ':', '-', ')', ':', '-P', '<', '3', 'and', 'some', 'arrows', '<', '>', '-', '>', '<', '--', '#', '@', '@', 'qq.com']
33
['lazycat.I', "'m", 'super', 'lying', 'man', ',', 'This', 'is', 'a', 'coooool', '#', 'dummysmiley', ':', ':', '-', ')', ':', '-P', '<', '3', 'and', 'some', 'arrows', '<', '>', '-', '>', '<', '--', '#', '@', '@', 'qq.com']
33
['lazycat', '.', "I'm", 'super', 'lying', 'man', ',', 'This', 'is', 'a', 'coooool', '#dummysmiley', ':', ':-)', ':-P', '<3', 'and', 'some', 'arrows', '<', '>', '->', '<--', '#', '@', '@qq', '.', 'com']
28
['lazycat', '.', "I'm", 'super', 'lying', 'man', ',', 'This', 'is', 'a', 'coool', '#dummysmiley', ':', ':-)', ':-P', '<3', 'and', 'some', 'arrows', '<', '>', '->', '<--', '#', '@', '@qq', '.', 'com']
28


In [150]:
def precessing_data(data):
    word = nltk.word_tokenize(data)
    return ' '.join(data)
#func_1
def change_x(data):
    data = data.map(precessing_data)
    return data

### ====================================================

### 分词： 法②

In [12]:
#先'/'再'-'再'.'再'com'
def getTokens(input):
    tokensBySlash = str(input.encode('utf-8')).split('/')	#get tokens after splitting by slash
    allTokens = []
    for i in tokensBySlash:
        tokens = str(i).split('-')	#get tokens after splitting by dash
        tokensByDot = []
        for j in range(0,len(tokens)):
            tempTokens = str(tokens[j]).split('.')	#get tokens after splitting by dot
            tokensByDot = tokensByDot + tempTokens
        allTokens = allTokens + tokens + tokensByDot
    allTokens = list(allTokens)#-remove_set--#remove redundant tokens
    if 'com' in allTokens:
        allTokens.remove('com')	#removing .com since it occurs a lot of times and it should not be included in our features
    return allTokens

In [13]:
def preprocessing_data_2(list_data):
    new_data = ''
    for i in list_data:
        words = nltk.tokenize.TweetTokenizer(strip_handles=False, reduce_len = True).tokenize(i)
        temp = ' '.join(words)
        new_data = new_data + ' '+temp
    return new_data

In [14]:
#test
url_test = 'https://www.bilibili.com/video/av4836703/fuckme home workplace&&>::-):-P<3@qq.com'
url_test = getTokens(url_test)
print(url_test)
print(len(url_test))

url_test = preprocessing_data_2(url_test)
print(url_test)
print(len(url_test))
#然后喂给Tfidf
# vectorizer = TfidfVectorizer(tokenizer=getTokens)	#get a vector for each url but use our customized tokenizer
# X = vectorizer.fit_transform(corpus)	#get the X vector

["b'https:", "b'https:", '', '', 'www.bilibili.com', 'www', 'bilibili', 'video', 'video', 'av4836703', 'av4836703', 'fuckme home workplace&&>::', '):', "P<3@qq.com'", 'fuckme home workplace&&>::', '):', 'P<3@qq', "com'"]
18
 b'https : b'https :   www.bilibili.com www bilibili video video av4836703 av4836703 fuckme home workplace & & >:: ): P <3 @qq . com ' fuckme home workplace & & >:: ): P <3 @qq com '
182


In [15]:
def pipeline_x(data):
    data = getTokens(data)
    data = preprocessing_data_2(data)
    return data

In [16]:
url_test = 'https://www.bilibili.com/video/av4836703/(fuck)me home workplace&&>::-):-P<3@qq.com'
url_test = pipeline_x(url_test)
print(url_test)

 b'https : b'https :   www.bilibili.com www bilibili video video av4836703 av4836703 ( fuck ) me home workplace & & >:: ): P <3 @qq . com ' ( fuck ) me home workplace & & >:: ): P <3 @qq com '


### ===============================================

## 处理数据X
思路：
- 按规则分词 + 特殊标记(TweetTokenizer)分词
- Tokenizer
- 喂给模型Embedding

In [17]:
pre_data = train_x.apply(lambda x:pipeline_x(x))

In [18]:
pre_data[0]

" b'diaryofagameaddict . com ' b'diaryofagameaddict com '"

In [19]:
content = ['', 'www.bilibili.com', 'video', '):', "com '", 'av4836703', "b'https :", 'fuckme home workplace & & >::', 'P <3 @qq', 'www', 'bilibili', "P <3 @qq . com '"]
t = ' '.join(content)
print(t)

 www.bilibili.com video ): com ' av4836703 b'https : fuckme home workplace & & >:: P <3 @qq www bilibili P <3 @qq . com '


In [20]:
# pre_dataa = []
# num = 0
# for i in pre_data:
#     for j in i:
#         pre_dataa.append(j)
#         num = num + 1
pre_dataa = []
for i in pre_data:
    temp = ' '.join(i)
    pre_dataa.append(temp)

In [21]:
len(pre_dataa)

420464

In [22]:
pre_data[0] #正确的

" b'diaryofagameaddict . com ' b'diaryofagameaddict com '"

In [23]:
pre_dataa[0]# 错误的

"  b ' d i a r y o f a g a m e a d d i c t   .   c o m   '   b ' d i a r y o f a g a m e a d d i c t   c o m   '"

In [24]:
print(train.shape[0])
print(pre_data.shape[0])

420464
420464


In [25]:
#处理文本为序列的整数列表，制作x，y
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np

maxlen = 15
training_sample = train.shape[0]
#validation_samples = 
max_words = 10000
#max_words个单词的文本转数值转换器
tokenizer = Tokenizer(num_words=max_words, lower=False)
tokenizer.fit_on_texts(pre_data)
#将字符串转化为整数索引的列表
sequences = tokenizer.texts_to_sequences(pre_data)#如果model=‘bianry’则是01表示
#获得  字符：数字 的字典
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))
#将列表转换为（samples，maxlen）的二维整数张量
data = pad_sequences(sequences, maxlen=maxlen)#maxlen设置最大的序列长度，长于该长度的序列将会截短，短于该长度的序列将会填充

Using TensorFlow backend.


Found 439245 unique tokens.


In [26]:
# import pickle
# f=open('./word_index.pk','wb')
# pickle.dump(word_index,f)

In [29]:
#f.close()

In [28]:
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', train_y.shape)

Shape of data tensor: (420464, 15)
Shape of label tensor: (420464,)


In [30]:
#处理完原始的sequence预览
sequences[1]

[2, 91, 1, 91, 1]

In [31]:
word_index

{"'": 1,
 'com': 2,
 'html': 3,
 'org': 4,
 'php': 5,
 'htm': 6,
 'index': 7,
 'net': 8,
 '1': 9,
 'wiki': 10,
 "b'en": 11,
 'wikipedia': 12,
 '2011': 13,
 'b': 14,
 'ca': 15,
 'aspx': 16,
 'id': 17,
 'the': 18,
 'of': 19,
 'v': 20,
 'watch': 21,
 'in': 22,
 "b'youtube": 23,
 'news': 24,
 'wp': 25,
 '2': 26,
 "b'facebook": 27,
 'blogspot': 28,
 'edu': 29,
 'co': 30,
 'en': 31,
 'people': 32,
 'and': 33,
 '2010': 34,
 'c': 35,
 'uk': 36,
 'info': 37,
 'yahoo': 38,
 'a': 39,
 'p': 40,
 's': 41,
 'login': 42,
 "b'amazon": 43,
 'asp': 44,
 'name': 45,
 '4': 46,
 'content': 47,
 'to': 48,
 '10': 49,
 'dp': 50,
 'fid': 51,
 '3': 52,
 'page': 53,
 'us': 54,
 '0': 55,
 '11': 56,
 'topic': 57,
 'view': 58,
 'article': 59,
 'directory': 60,
 'pages': 61,
 '2009': 62,
 'de': 63,
 '18': 64,
 "b'imdb": 65,
 'n': 66,
 "b'linkedin": 67,
 'includes': 68,
 'pub': 69,
 'search': 70,
 'city': 71,
 'images': 72,
 'm': 73,
 'cgi': 74,
 'ru': 75,
 'admin': 76,
 'music': 77,
 't': 78,
 'gov': 79,
 'br': 80,


In [32]:
word_counts = tokenizer.word_counts

In [33]:
word_counts

OrderedDict([("b'diaryofagameaddict", 2),
             ('com', 319223),
             ("'", 859835),
             ("b'espdesign", 2),
             ('au', 5796),
             ("b'iamagameaddict", 2),
             ("b'kalantzis", 2),
             ('net', 34048),
             ("b'slightlyoffcenter", 2),
             ("b'toddscarwash", 2),
             ("b'tubemoviez", 2),
             ("b'ipl", 14),
             ('hk', 218),
             ("b'crackspider", 2),
             ('us', 8774),
             ('toolbar', 30),
             ('install', 270),
             ('php', 70441),
             ('pack', 122),
             ('exe', 4390),
             ("b'pos", 10),
             ('kupang', 2),
             ("b'rupor", 2),
             ('info', 11829),
             ("b'svision", 2),
             ('online', 3142),
             ('de', 7248),
             ('mgfi', 2),
             ('administrator', 710),
             ('components', 1294),
             ('babackup', 2),
             ('classes', 232),
    

In [34]:
#对数据进行分割
#先打乱
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = train_y[indices]

train_sample = data.shape[0]  - (data.shape[0] // 10)
print('sum',data.shape[0])
print('indexs',train_sample)

x_train = data[:train_sample]
y_train = labels[:train_sample]
x_val = data[train_sample:]
y_val = labels[train_sample:]

sum 420464
indexs 378418


In [35]:
print(x_train.shape)
print(x_val.shape)

(378418, 15)
(42046, 15)


In [36]:
x_train[:10]

array([[3493,  445,  445,  148, 7205,  148, 7205, 7567, 7567,  315,   42,
         315,   42,    1,    1],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,  733,
           2,  733,    1,    1],
       [   0,    0,    0,    0,    0,    0,    0,    0,  233,    2,  233,
          37,   37,    1,    1],
       [1061,    2, 1061,  110,  110, 1853,  502,  502, 3213,    1, 1853,
         502,  502, 3213,    1],
       [   0,    0,    0,    0,    0,    0,    0,    0,  913,    2,  913,
         256,    1,  256,    1],
       [   0,    0,    0,    0,    0,    0,    0,    0,    2,  623,  623,
         658,    1,  658,    1],
       [7406,   33, 3581, 7406, 3130,  170,    3,    1,   33, 3581, 7406,
        3130,  170,    3,    1],
       [1638, 1638,  198,  198,  375,  375,   87,  562,    2,   87,  562,
         138,    1,  138,    1],
       [   0,    0,   27,    2,   27,   61,   61,  117, 5096, 2873,  117,
        5096, 2873,    1,    1],
       [   0,    0,    0,    0,    0,

In [37]:
import os
GLOVE_DIR = 'D:\Jupyter\glove.6B'
embeddings_index = {}
with open(os.path.join(GLOVE_DIR, 'glove.6B.50d.txt'), encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [38]:
EMBEDDING_DIM = 50
embedding_matrix = np.zeros((len(word_index)+1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

### =================================================

### 版本一：使用Glove

In [250]:
from keras import layers

rate_drop_lstm = 0.2

#【max_words单词， embedding_dims单词对应的维度向量】
embedding_dim = 100


# 定义模型
#【样本，每个样本100单词，每个单词100维度】
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense, Bidirectional, LSTM

model = Sequential()
model.add(Embedding(len(word_index)+1,
                    EMBEDDING_DIM,
                    input_length=maxlen,
                    weights=[embedding_matrix],
                    trainable=False,
                   ))#构建词嵌入，每个单词
# model.add(layers.Conv1D(32, 5, activation='relu'))
# model.add(layers.MaxPooling1D(3))
model.add(LSTM(64, dropout=rate_drop_lstm, recurrent_dropout=rate_drop_lstm))
#model.add(Flatten())
model.add(Dense(32, activation='relu'))#负责单词之间的练习和语义
model.add(Dense(1, activation='sigmoid'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 15, 50)            21962300  
_________________________________________________________________
lstm_7 (LSTM)                (None, 64)                29440     
_________________________________________________________________
dense_13 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_14 (Dense)             (None, 1)                 33        
Total params: 21,993,853
Trainable params: 31,553
Non-trainable params: 21,962,300
_________________________________________________________________


In [251]:
model.compile(optimizer='rmsprop',
             loss='binary_crossentropy',
             metrics=['acc'])
history = model.fit(x_train, y_train,
                   epochs=10,
                   batch_size=32,
                   validation_data=(x_val, y_val),
                   #callbacks=callback_list,
                    verbose =1)

Train on 378418 samples, validate on 42046 samples
Epoch 1/10
378418/378418 [==============================] - 348s 919us/step - loss: 0.2146 - acc: 0.9177 - val_loss: 0.1560 - val_acc: 0.9415
Epoch 2/10
378418/378418 [==============================] - 343s 906us/step - loss: 0.1719 - acc: 0.9360 - val_loss: 0.1491 - val_acc: 0.9462
Epoch 3/10
378418/378418 [==============================] - 338s 893us/step - loss: 0.1633 - acc: 0.9403 - val_loss: 0.1434 - val_acc: 0.9484
Epoch 4/10
378418/378418 [==============================] - 338s 892us/step - loss: 0.1602 - acc: 0.9420 - val_loss: 0.1381 - val_acc: 0.9503
Epoch 5/10
378418/378418 [==============================] - 335s 885us/step - loss: 0.1585 - acc: 0.9424 - val_loss: 0.1398 - val_acc: 0.9499
Epoch 6/10
378418/378418 [==============================] - 337s 890us/step - loss: 0.1593 - acc: 0.9426 - val_loss: 0.1368 - val_acc: 0.9511
Epoch 7/10
378418/378418 [==============================] - 332s 876us/step - loss: 0.1592 - acc:

In [253]:
model.save('used_Glove.h5')
model.save_weights('used_Glove_weight.h5')

### 版本二：自训练Embedding，效果不好
如果解决又取max_words又能导入embedding_weight就好了

In [220]:
from keras import layers

rate_drop_lstm = 0.2

#【max_words单词， embedding_dims单词对应的维度向量】
embedding_dim = 100


# 定义模型
#【样本，每个样本100单词，每个单词100维度】
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense, Bidirectional, LSTM

model_ = Sequential()
model_.add(Embedding(max_words,
                    EMBEDDING_DIM,
                    input_length=maxlen,
                   ))#构建词嵌入，每个单词
# model.add(layers.Conv1D(32, 5, activation='relu'))
# model.add(layers.MaxPooling1D(3))
model_.add(LSTM(64, dropout=rate_drop_lstm, recurrent_dropout=rate_drop_lstm))
#model.add(Flatten())
model_.add(Dense(32, activation='relu'))#负责单词之间的练习和语义
model_.add(Dense(1, activation='sigmoid'))
model_.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 15, 50)            500000    
_________________________________________________________________
lstm_6 (LSTM)                (None, 64)                29440     
_________________________________________________________________
dense_11 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 33        
Total params: 531,553
Trainable params: 531,553
Non-trainable params: 0
_________________________________________________________________


In [221]:
model.compile(optimizer='rmsprop',
             loss='binary_crossentropy',
             metrics=['acc'])
history = model.fit(x_train, y_train,
                   epochs=10,
                   batch_size=32,
                   validation_data=(x_val, y_val),
                   #callbacks=callback_list,
                    verbose =1)

Train on 378418 samples, validate on 42046 samples
Epoch 1/10
325952/378418 [========================>.....] - ETA: 46s - loss: 0.4738 - acc: 0.8195

KeyboardInterrupt: 

### 版本三 ： 加入Attention

In [38]:
from keras import layers
from keras.layers import merge
from keras.models import *
from keras.layers import Embedding, Flatten, Dense, Bidirectional, LSTM
from keras.layers.core import *
#[batch_size, ax_words]
#【max_words单词， embedding_dims单词对应的维度向量】
embedding_dim = 50
rate_drop_lstm = 0.2
SINGLE_ATTENTION_VECTOR = True

def attention_3d_block(inputs):
    # inputs.shape = (batch_size, time_steps15, input_dim50)
    print(inputs.shape)
    input_dim = int(inputs.shape[2])
    time_steps = int(inputs.shape[1])
    
    a = Permute((2, 1))(inputs)
    #a = Reshape((input_dim, TIME_STEPS))(a) # this line is not useful. It's just to know which dimension is what.
    a = Dense(time_steps, activation='softmax')(a)
    if SINGLE_ATTENTION_VECTOR:
        a = Lambda(lambda x: K.mean(x, axis=1), name='dim_reduction')(a)
        a = RepeatVector(input_dim)(a)
    a_probs = Permute((2, 1), name='attention_vec')(a)
    output_attention_mul = merge([inputs, a_probs], name='attention_mul', mode='mul')
    return output_attention_mul



# 定义模型
#【样本，每个样本100单词，每个单词100维度】
# EMBEDDING_DIM = 50
# maxlen = 15

inputs = Input(shape=(maxlen,))
# 层输入【samples， maxlen】
# 层输出【samples， maxlen， dims】
emb = Embedding(len(word_index)+1,
                    EMBEDDING_DIM,
                    input_length=maxlen,
                    weights=[embedding_matrix],
                    trainable=False,
                   )#构建词嵌入，每个单词
emb = emb(inputs)
attention_mul = attention_3d_block(emb)
# model.add(layers.Conv1D(32, 5, activation='relu'))
# model.add(layers.MaxPooling1D(3))
x = LSTM(64, dropout=rate_drop_lstm, recurrent_dropout=rate_drop_lstm)(emb)
#model.add(Flatten())
x = Dense(32, activation='relu')(x)#负责单词之间的练习和语义
output = Dense(1, activation='sigmoid')(x)
model = Model(inputs, output)
model.summary()

(?, 15, 50)
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


F:\Anaconda\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:25: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
F:\Anaconda\envs\tensorflow-gpu\lib\site-packages\keras\legacy\layers.py:458: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 15)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 15, 50)            21962300  
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                29440     
_________________________________________________________________
dense_2 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 33        
Total params: 21,993,853
Trainable params: 31,553
Non-trainable params: 21,962,300
_________________________________________________________________


In [40]:
from keras.callbacks import TensorBoard, ReduceLROnPlateau, EarlyStopping, ModelCheckpoint


class Callback(object):
    def __init__(self, model_name):
        self.model_name = model_name
        self.log_dir = "{}_logs/".format(self.model_name)
        self.model_dir = "{}_models/".format(self.model_name)

    def get_tensorboard(self):#是一个可视化的展示器
        tensorboard_callback = TensorBoard(log_dir=self.log_dir, write_grads=True,
                                           histogram_freq=0, write_images=True)
        return tensorboard_callback

    def get_early_stop(self, patience):#当监测值不再改善时，该回调函数将中止训练
        early_stop = EarlyStopping('val_acc', patience=patience)
        return early_stop

    def get_readuce_lr(self, factor, patience):#学习率衰减
        return ReduceLROnPlateau(monitor='val_acc', factor=factor, patience=patience)

    def get_model_ckpt(self):#在每个epoch后保存模型
        model_names = self.model_dir + '.{epoch:02d}-{val_acc:.4f}.h5'
        model_checkpoint = ModelCheckpoint(model_names, monitor='val_acc', verbose=1, save_best_only=True)
        return model_checkpoint

callbacks = Callback('LSTM')
tensorboard = callbacks.get_tensorboard()
early_stop = callbacks.get_early_stop(5)
readuce_lr = callbacks.get_readuce_lr(factor=0.5, patience=50 // 4)
model_ckpt = callbacks.get_model_ckpt()
callback_list = [tensorboard, early_stop, readuce_lr]

In [41]:
model.compile(optimizer='rmsprop',
             loss='binary_crossentropy',
             metrics=['acc'])
history = model.fit(x_train, y_train,
                   epochs=10,
                   batch_size=32,
                   validation_data=(x_val, y_val),
                   #callbacks=callback_list,
                    verbose =1)

Train on 378418 samples, validate on 42046 samples
Epoch 1/10
 23456/378418 [>.............................] - ETA: 6:56 - loss: 0.3343 - acc: 0.8666

KeyboardInterrupt: 

In [38]:
model.save('used_Attention_LSTM.h5')
model.save_weights('used_attention_LSTM_weight.h5')

### 对比实验，使用RNN

In [39]:
from keras import layers

rate_drop_lstm = 0.2

#【max_words单词， embedding_dims单词对应的维度向量】
embedding_dim = 100


# 定义模型
#【样本，每个样本100单词，每个单词100维度】
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense, Bidirectional, LSTM

model = Sequential()
model.add(Embedding(len(word_index)+1,
                    EMBEDDING_DIM,
                    input_length=maxlen,
                    weights=[embedding_matrix],
                    trainable=False,
                   ))#构建词嵌入，每个单词
# model.add(layers.Conv1D(32, 5, activation='relu'))
# model.add(layers.MaxPooling1D(3))
model.add(layers.SimpleRNN(64))
#model.add(Flatten())
model.add(Dense(32, activation='relu'))#负责单词之间的练习和语义
model.add(Dense(1, activation='sigmoid'))
model.summary()

Instructions for updating:
keep_dims is deprecated, use keepdims instead
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 15, 50)            21962300  
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 64)                7360      
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 33        
Total params: 21,971,773
Trainable params: 9,473
Non-trainable params: 21,962,300
_________________________________________________________________


In [40]:
model.compile(optimizer='rmsprop',
             loss='binary_crossentropy',
             metrics=['acc'])
history = model.fit(x_train, y_train,
                   epochs=4,
                   batch_size=32,
                   validation_data=(x_val, y_val),
                   #callbacks=callback_list,
                    verbose =1)

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Train on 378418 samples, validate on 42046 samples
Epoch 1/4
378418/378418 [==============================] - 140s 369us/step - loss: 0.2185 - acc: 0.9165 - val_loss: 0.1929 - val_acc: 0.9317
Epoch 2/4
378418/378418 [==============================] - 136s 361us/step - loss: 0.1799 - acc: 0.9336 - val_loss: 0.1732 - val_acc: 0.9371
Epoch 3/4
378418/378418 [==============================] - 135s 357us/step - loss: 0.1701 - acc: 0.9384 - val_loss: 0.1787 - val_acc: 0.9390
Epoch 4/4
378418/378418 [==============================] - 143s 377us/step - loss: 0.1665 - acc: 0.9401 - val_loss: 0.1764 - val_acc: 0.9399


### 对比实验，使用GRU

In [41]:
from keras import layers

rate_drop_lstm = 0.2

#【max_words单词， embedding_dims单词对应的维度向量】
embedding_dim = 100


# 定义模型
#【样本，每个样本100单词，每个单词100维度】
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense, Bidirectional, LSTM

model = Sequential()
model.add(Embedding(len(word_index)+1,
                    EMBEDDING_DIM,
                    input_length=maxlen,
                    weights=[embedding_matrix],
                    trainable=False,
                   ))#构建词嵌入，每个单词
# model.add(layers.Conv1D(32, 5, activation='relu'))
# model.add(layers.MaxPooling1D(3))
model.add(layers.GRU(64))
#model.add(Flatten())
model.add(Dense(32, activation='relu'))#负责单词之间的练习和语义
model.add(Dense(1, activation='sigmoid'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 15, 50)            21962300  
_________________________________________________________________
gru_1 (GRU)                  (None, 64)                22080     
_________________________________________________________________
dense_3 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 33        
Total params: 21,986,493
Trainable params: 24,193
Non-trainable params: 21,962,300
_________________________________________________________________


In [42]:
model.compile(optimizer='rmsprop',
             loss='binary_crossentropy',
             metrics=['acc'])
history = model.fit(x_train, y_train,
                   epochs=4,
                   batch_size=32,
                   validation_data=(x_val, y_val),
                   #callbacks=callback_list,
                    verbose =1)

Train on 378418 samples, validate on 42046 samples
Epoch 1/4
378418/378418 [==============================] - 329s 871us/step - loss: 0.1824 - acc: 0.9310 - val_loss: 0.1564 - val_acc: 0.9423
Epoch 2/4
378418/378418 [==============================] - 247s 653us/step - loss: 0.1416 - acc: 0.9480 - val_loss: 0.1423 - val_acc: 0.9491
Epoch 3/4
378418/378418 [==============================] - 236s 623us/step - loss: 0.1315 - acc: 0.9521 - val_loss: 0.1343 - val_acc: 0.9520
Epoch 4/4
378418/378418 [==============================] - 252s 666us/step - loss: 0.1270 - acc: 0.9537 - val_loss: 0.1326 - val_acc: 0.9530


### 对比实验，使用双向LSTM

In [43]:
from keras import layers

rate_drop_lstm = 0.2

#【max_words单词， embedding_dims单词对应的维度向量】
embedding_dim = 100


# 定义模型
#【样本，每个样本100单词，每个单词100维度】
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense, Bidirectional, LSTM

model = Sequential()
model.add(Embedding(len(word_index)+1,
                    EMBEDDING_DIM,
                    input_length=maxlen,
                    weights=[embedding_matrix],
                    trainable=False,
                   ))#构建词嵌入，每个单词
# model.add(layers.Conv1D(32, 5, activation='relu'))
# model.add(layers.MaxPooling1D(3))
model.add(layers.Bidirectional(LSTM(64, dropout=rate_drop_lstm, recurrent_dropout=rate_drop_lstm)))
#model.add(Flatten())
model.add(Dense(32, activation='relu'))#负责单词之间的练习和语义
model.add(Dense(1, activation='sigmoid'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 15, 50)            21962300  
_________________________________________________________________
bidirectional_1 (Bidirection (None, 128)               58880     
_________________________________________________________________
dense_5 (Dense)              (None, 32)                4128      
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 33        
Total params: 22,025,341
Trainable params: 63,041
Non-trainable params: 21,962,300
_________________________________________________________________


In [44]:
model.compile(optimizer='rmsprop',
             loss='binary_crossentropy',
             metrics=['acc'])
history = model.fit(x_train, y_train,
                   epochs=4,
                   batch_size=32,
                   validation_data=(x_val, y_val),
                   #callbacks=callback_list,
                    verbose =1)

Train on 378418 samples, validate on 42046 samples
Epoch 1/4
378418/378418 [==============================] - 863s 2ms/step - loss: 0.2079 - acc: 0.9210 - val_loss: 0.1565 - val_acc: 0.9408
Epoch 2/4
378418/378418 [==============================] - 740s 2ms/step - loss: 0.1672 - acc: 0.9382 - val_loss: 0.1498 - val_acc: 0.9457
Epoch 3/4
378418/378418 [==============================] - 626s 2ms/step - loss: 0.1583 - acc: 0.9422 - val_loss: 0.1438 - val_acc: 0.9487
Epoch 4/4
378418/378418 [==============================] - 595s 2ms/step - loss: 0.1546 - acc: 0.9438 - val_loss: 0.1366 - val_acc: 0.9505
